## # **Data Cleaning & Ad Hoc Analysis with SQL**

### What we will see and what data Analyst require in real life



*   Playing with Date column
*   Findings/Removing Duplicate
*   Handalling/Imputing/Replacing missing values
*   Standardize Categorical Data
*   Filtering Out
*   Flatten Nested Field
*   Combine/Concatenet data
*   Conditional Columns
*   RegEx using
*   Creating views



## ## **1. Convert date to Proper Date Format**

Description: Convert the integer date field to a readable DATE format.


```
SELECT
  PARSE_DATE('%Y%m%d', CAST(date AS STRING)) AS formatted_date
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`;

```





## **Date functions everything in one query**

```
SELECT
  date,
  PARSE_DATE('%Y%m%d', CAST(date AS STRING)) AS formatted_date,
  FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', TIMESTAMP_SECONDS(visitStartTime)) AS formatted_time,
  EXTRACT(YEAR FROM TIMESTAMP_SECONDS(visitStartTime)) AS year,
  EXTRACT(MONTH FROM TIMESTAMP_SECONDS(visitStartTime)) AS month,
  EXTRACT(DAY FROM TIMESTAMP_SECONDS(visitStartTime)) AS day,
  EXTRACT(DAYOFWEEK FROM TIMESTAMP_SECONDS(visitStartTime)) AS day_of_week,  -- 1 = Sunday, 7 = Saturday
  EXTRACT(HOUR FROM TIMESTAMP_SECONDS(visitStartTime)) AS hour,
  EXTRACT(MINUTE FROM TIMESTAMP_SECONDS(visitStartTime)) AS minute,
  CASE
    WHEN EXTRACT(HOUR FROM TIMESTAMP_SECONDS(visitStartTime)) BETWEEN 0 AND 6 THEN 'Late Night'
    WHEN EXTRACT(HOUR FROM TIMESTAMP_SECONDS(visitStartTime)) BETWEEN 7 AND 12 THEN 'Morning'
    WHEN EXTRACT(HOUR FROM TIMESTAMP_SECONDS(visitStartTime)) BETWEEN 13 AND 18 THEN 'Afternoon'
    ELSE 'Evening'
  END AS time_period,
  CASE
    WHEN EXTRACT(DAYOFWEEK FROM TIMESTAMP_SECONDS(visitStartTime)) IN (1, 7) THEN 'Weekend'
    ELSE 'Weekday'
  END AS day_type,
  COUNT(*) AS visit_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  date,year, month, day, day_of_week, hour, minute, formatted_time, time_period, day_type
LIMIT 10;

```

## ## **2. Findings Duplicate Rows diff ways**

Description: Ensure no duplicate combinations of fullVisitorId and visitId exist.

Using COUNT with GROUP BY


```
SELECT
  fullVisitorId,
  COUNT(*) AS duplicate_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  fullVisitorId
HAVING
  COUNT(*) > 1
ORDER BY
  duplicate_count DESC;

```



Using COUNTIF()



```
SELECT
  fullVisitorId,
  COUNTIF(fullVisitorId = fullVisitorId) AS duplicate_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  fullVisitorId
HAVING
  duplicate_count > 1;

```



Using ROW_NUMBER() Window Function



```
SELECT
  *
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY fullVisitorId ORDER BY visitId) AS row_num
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
)
WHERE
  row_num > 1
ORDER BY
  fullVisitorId, visitId;

```



## ## **3. Duplicate Ignoring**

Using DISTINCT to Remove Duplicate Rows



```
SELECT DISTINCT
  *
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;

```



Removing Duplicates Based on Specific Columns



```
SELECT
  *
FROM (
  SELECT
    *,
    ROW_NUMBER() OVER (PARTITION BY fullVisitorId ORDER BY visitId) AS row_num
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
)
WHERE
  row_num = 1;

```



## ## **4. Missing Values**

Description: Replace NULL or empty totals.
transactionRevenue fiels with "0".


**Checking Missing Values**
```
SELECT
  COUNTIF(totals.transactionRevenue IS NULL) AS null_count,
  COUNTIF(totals.transactionRevenue IS NOT NULL) AS not_null_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;
```




**Handaling/Imputing Missing Revenue**
```
SELECT
  COALESCE(totals.transactionRevenue, 0) AS revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;


```


```
SELECT
  trafficSource.referralPath,
  IFNULL(trafficSource.referralPath, "Direct") AS referralPath
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;
```





## ## **5. Standardize Categorical Data**

Description: Convert trafficSource to uppercase for consistency.



```
SELECT
  trafficSource.source,
  UPPER(trafficSource.source) AS source_uppercase
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
LIMIT 10;


```



## ## **6. Filter Out data**

Description: Exclude sessions with pageviews exceeding a defined threshold (e.g., 1000).



```
SELECT *
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  totals.pageviews <= 1000;

```



```
SELECT *
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
WHERE
  trafficSource.source LIKE "%google%" OR trafficSource.source LIKE "%bing%";
```





## ## **7. Flatten Nested hits Field**

Description: Unnest hits for detailed event-level data.



```
SELECT
  fullVisitorId,
  visitId,
  hits.page.pagePath,
  hits.eventInfo.eventCategory
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`,
  UNNEST(hits) AS hits;
```



## ## **8. Identify and Handle Outliers in Session Duration**

Description: Flag sessions with abnormally long durations.
sql



```
SELECT *
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  totals.timeOnSite > 3600; -- More than an hour

```



## ## **9. Combine Source and Medium**

Description: Concatenate source and medium for better analysis.



```
SELECT
  CONCAT(trafficSource.source, " / ", trafficSource.medium) AS source_medium
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`;

```



## ## **10. Conditional Columns**

Description: Flag sessions with only one hit as bounces.



```
SELECT
  CASE WHEN totals.pageviews = 1 THEN TRUE ELSE FALSE END AS is_bounce
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;

```






## ## **11. Standardize Device Categories**
Description: Group devices into broader categories (e.g., Mobile, Desktop, Tablet).



```
SELECT
  distinct device.deviceCategory,
  CASE
    WHEN device.deviceCategory = "mobile" THEN "Mobile"
    WHEN device.deviceCategory = "tablet" THEN "Tablet"
    ELSE "Desktop"
  END AS device_type
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;

```



## ## **12. Convert Currency Fields**

Description: Convert revenue fields from strings to numbers for calculations.



```
SELECT
  CAST(totals.transactionRevenue AS FLOAT64) AS revenue
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`;

```



## ## **13. Remove Special Characters**

You might want to remove special characters or punctuation that are not relevant to the analysis. This can be done using regular expressions (REGEXP_REPLACE).



```
SELECT
  trafficSource.source,
  REGEXP_REPLACE(trafficSource.source, r'[^a-zA-Z0-9\s]', '') AS clean_source
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
LIMIT 10;

```



## ## **14. Add a Calculated Column for Session Value**

Description: Derive a metric for the value of each session.



```
SELECT
  PARSE_DATE('%Y%m%d', CAST(date AS STRING)) AS formatted_date,
  SUM(CAST(totals.transactionRevenue AS FLOAT64) / totals.visits) AS session_value
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170101' AND '20170131'
AND
  totals.visits > 0
GROUP BY formatted_date

```


```
SELECT
  SUM(totals.pageviews) AS total_pageviews,
  SUM(CAST(totals.transactionRevenue AS FLOAT64)) AS total_revenue,
  COUNT(DISTINCT fullVisitorId) AS unique_visitors
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
  _TABLE_SUFFIX BETWEEN '20170101' AND '20170131';
```





# SQL Script: Data Cleaning and Structuring & saving as views for further use of Regression analysis



```
CREATE VIEW `keen-phalanx-396514.Assignment.ga4agg` AS
WITH cleaned_data AS (
  SELECT
    -- Visitor and session identifiers
    fullVisitorId,
    visitId,

    -- Date field as a proper DATE type
    PARSE_DATE('%Y%m%d', CAST(date AS STRING)) AS visit_date,

    -- Numeric metrics with NULLs replaced by 0
    COALESCE(CAST(totals.pageviews AS INT64), 0) AS pageviews,
    COALESCE(CAST(totals.hits AS INT64), 0) AS hits,
    COALESCE(CAST(totals.transactions AS INT64), 0) AS transactions,
    COALESCE(CAST(totals.transactionRevenue AS FLOAT64), 0.0) AS revenue,  -- Updated to use transactionRevenue
    COALESCE(CAST(totals.timeOnSite AS INT64), 0) AS time_on_site,

    -- Traffic source (standardized and combined)
    LOWER(CONCAT(trafficSource.source, " / ", trafficSource.medium)) AS source_medium,

    -- Device category (simplified to Mobile/Desktop/Tablet)
    CASE
      WHEN device.deviceCategory = "mobile" THEN "Mobile"
      WHEN device.deviceCategory = "tablet" THEN "Tablet"
      ELSE "Desktop"
    END AS device_category,

    -- Geographic information with NULLs replaced by "Unknown"
    COALESCE(geoNetwork.country, "Unknown") AS country,
    COALESCE(geoNetwork.city, "Unknown") AS city,

    -- Bounce rate indicator
    CASE WHEN COALESCE(CAST(totals.pageviews AS INT64), 0) = 1 THEN TRUE ELSE FALSE END AS is_bounce,

    -- Derived session value
    CASE
      WHEN COALESCE(CAST(totals.transactions AS INT64), 0) > 0
           AND COALESCE(CAST(totals.transactionRevenue AS FLOAT64), 0.0) > 0  -- Updated for transactionRevenue
      THEN CAST(totals.transactionRevenue AS FLOAT64) / CAST(totals.transactions AS FLOAT64)
      ELSE 0.0
    END AS session_value
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
  WHERE
    _TABLE_SUFFIX BETWEEN '20170101' AND '20170131' -- Filter data for January 2017
    AND fullVisitorId IS NOT NULL -- Remove rows with missing visitor IDs
),

-- STEP 2: Aggregate key metrics by visitor for regression
aggregated_data AS (
  SELECT
    fullVisitorId,
    COUNT(DISTINCT visitId) AS session_count,
    SUM(pageviews) AS total_pageviews,
    SUM(hits) AS total_hits,
    SUM(transactions) AS total_transactions,
    SUM(revenue) AS total_revenue,
    AVG(time_on_site) AS avg_time_on_site,
    AVG(session_value) AS avg_session_value,
    COUNTIF(is_bounce) AS total_bounces,
    -- Device and source statistics
    ANY_VALUE(device_category) AS primary_device_category,
    ANY_VALUE(source_medium) AS primary_source_medium,
    -- Geographic details
    ANY_VALUE(country) AS primary_country
  FROM
    cleaned_data
  GROUP BY
    fullVisitorId
),

-- STEP 3: Prepare the final dataset for regression
final_data AS (
  SELECT
    *,
    -- Calculate bounce rate as a percentage
    (total_bounces * 100.0 / session_count) AS bounce_rate,
    -- Log-transform revenue for regression purposes
    LOG(NULLIF(total_revenue, 0) + 1) AS log_revenue
  FROM
    aggregated_data
)

-- STEP 4: Output the final dataset
SELECT *
FROM final_data


```



**Explanation of Steps:**
Cleaning the Data (CTE: cleaned_data):

This step handles missing values and converts fields into usable formats:
Date conversion ensures proper date formatting.
Numeric fields like pageviews, hits, and transactions are cleaned with COALESCE to replace NULL values with 0, ensuring consistency for analysis.
Categorical fields like country, city, and deviceCategory are standardized, replacing NULL values with default labels like "Unknown" or consolidating values like mobile, tablet, and desktop into three categories.
Bounce rate is calculated by marking sessions with only one pageview as a bounce, allowing us to identify low-engagement sessions.
A derived session value is computed by dividing revenue by transactions, highlighting the value of each session.
Aggregation (CTE: aggregated_data):

This step groups the data by fullVisitorId and aggregates key metrics:
Session count, total pageviews, hits, and transactions provide a high-level summary of user activity.
Total revenue and average session value are derived to understand user spending and behavior.
Bounce count is aggregated to calculate bounce rate.
Device and source statistics capture the most common device used and the source/medium combination for each visitor.
Geographic information is consolidated to provide a primary country for each visitor.
Feature Engineering (CTE: final_data):

This step prepares the data for regression analysis:
Bounce rate is calculated as a percentage to reflect the proportion of sessions that resulted in a bounce.
Log-transformed revenue is used to make the revenue data more suitable for regression models by stabilizing variance and ensuring a more normal distribution.
Final Output:

The final_data CTE produces the cleaned, aggregated, and feature-engineered dataset. The final output includes:
Visitor-level metrics like total_pageviews, total_hits, total_revenue, etc.
Derived fields like bounce_rate and log_revenue for regression analysis.
Standardized categorical data for device type, source/medium, and country.
This dataset is now ready for regression modeling, with numeric and categorical variables that will provide insights into user behavior and potential revenue predictions.

## # **Example Queries Using Relevant Columns**

1. Sessions by Device Category and Browser



```
SELECT
  device.deviceCategory,
  device.browser,
  COUNT(*) AS session_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  device.deviceCategory,
  device.browser
ORDER BY
  session_count DESC
LIMIT 10;

```



2. Top Countries by Page views



```
SELECT
  geoNetwork.country,
  SUM(totals.pageviews) AS total_pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
WHERE
 totals.pageviews IS NOT NULL
GROUP BY
  geoNetwork.country
order by
total_pageviews desc
limit 10

```



3. Pageviews Distribution by Traffic Source



```
SELECT
  trafficSource.source,
  trafficSource.medium,
  AVG(totals.pageviews) AS avg_pageviews,
  SUM(totals.pageviews) AS total_pageviews
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  trafficSource.source, trafficSource.medium
ORDER BY
  total_pageviews DESC
LIMIT 10;

```



4. Analyze Bounce Rates by Device



```
SELECT
  device.deviceCategory,
  COUNTIF(totals.bounces = 1) / COUNT(*) AS bounce_rate
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  device.deviceCategory
ORDER BY
  bounce_rate DESC;

```



5. Product Sales Analysis

```
SELECT
  product.v2ProductName AS product_name,
  SUM(product.productQuantity) AS total_quantity
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`,
  UNNEST(hits) AS hit,
  UNNEST(hit.product) AS product
WHERE
  product.v2ProductName IS NOT NULL
GROUP BY
  product_name
ORDER BY
  total_quantity DESC
LIMIT 10;

```



6. Hourly Traffic Analysis


```
SELECT
  EXTRACT(HOUR FROM TIMESTAMP_SECONDS(visitStartTime)) AS hour,
  COUNT(*) AS session_count
FROM
  `bigquery-public-data.google_analytics_sample.ga_sessions_20170801`
GROUP BY
  hour
ORDER BY
  hour;

```



# Conclusion
This notebook demonstrates a comprehensive approach to data cleaning using SQL. Each step ensures the data is prepared for advanced analysis, including regression and reporting.